# Проект по спринту "Машинное обучение в бизнесе" 

**Описание исследования**
Представим, что мы работаем в добывающей компании "ГлавРосГосНефть", и нам нужно решить, где бурить скважину. У нас имеются данные о пробах нефти в трех регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель МО, которая поможет определить регион, где добыча принесет наибольшую прибыль, и проанализировать возможную прибыль и риски с техникой Bootstrap.

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Описание данных**

Данные геологоразведки трёх регионов находятся в файлах: 
- `/datasets/geo_data_0.csv` 
- `/datasets/geo_data_1.csv`
- `/datasets/geo_data_2.csv`
- `id` — уникальный идентификатор скважины;
- `f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

<a class="anchor" id="chapter1"></a>
## Импортируем библиотеки

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

RANDOM_STATE = 42
TEST_SIZE = 0.25

<a class="anchor" id="chapter2"></a>
## Загрузка и подготовка данных

In [3]:
try: 
    geo0_data = pd.read_csv('/datasets/geo_data_0.csv')
    geo1_data = pd.read_csv('/datasets/geo_data_1.csv')
    geo2_data = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo0_data = pd.read_csv('geo_data_0.csv')
    geo1_data = pd.read_csv('geo_data_1.csv')
    geo2_data = pd.read_csv('geo_data_2.csv')

In [4]:
lst = [geo0_data, geo1_data, geo2_data]
for i in lst:
    display(i.head(3))
    display(i.info())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
print(geo0_data.duplicated().sum())
print(geo1_data.duplicated().sum())
print(geo2_data.duplicated().sum())

0
0
0


Загрузили три csv-файла в переменные, вывели несколько строчек из каждой таблицы и посмотрели на общую информации о них. Видим, что пропущенных значений нет, как и дубликатов. 

<a class="anchor" id="chapter3"></a>
## Обучение и проверка модели

Разделим каждый csv-файл на входные и целевые признаки, затем на обучающую и валидационную выборки. Обучим модель Линейной регрессии и посчитаем метрику RMSE.

In [5]:
def train_model(data):
    X = data.drop(columns=['product', 'id'])
    y = data['product']
    X_train, X_val, y_train, y_val = train_test_split(
        X,
        y,
        test_size = TEST_SIZE, 
        random_state = RANDOM_STATE)

    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_val)
    rmse_score = mean_squared_error(y_val, predictions, squared=False)
    return predictions, y_val, rmse_score

predictions_0, y_val0, rmse_score0 = train_model(geo0_data)
predictions_1, y_val1, rmse_score1 = train_model(geo1_data)
predictions_2, y_val2, rmse_score2 = train_model(geo2_data)

print('RMSE geo0_data:', rmse_score0)
print('RMSE geo1_data:', rmse_score1)
print('RMSE geo2_data:', rmse_score2)
print()
print('Mean predictions geo0_data:', predictions_0.mean())
print('Mean predictions geo1_data:', predictions_1.mean())
print('Mean predictions geo2_data:', predictions_2.mean())

RMSE geo0_data: 37.75660035026169
RMSE geo1_data: 0.890280100102884
RMSE geo2_data: 40.145872311342174

Mean predictions geo0_data: 92.39879990657768
Mean predictions geo1_data: 68.71287803913762
Mean predictions geo2_data: 94.77102387765939


In [10]:
results_0 = pd.DataFrame({'product': y_val0, 'preds': predictions_0})

results_1 = pd.DataFrame({'product': y_val1, 'preds': predictions_1})

results_2 = pd.DataFrame({'product': y_val2, 'preds': predictions_2})

Результаты показывают, что модель линейной регрессии, обученная на данных `geo1_data`, имеет наилучшее качество предсказания, имея более низкое значение RMSE (0.89) по сравнению с `geo0_data` (37.76) и `geo2_data` (40.15). Это означает, что предсказания для `geo1_data` гораздо ближе к истинным значениям, чем для двух других регионов.

## Подготовка к расчёту прибыли

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.

- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Расчет безубыточного объема сырья:

* Стоимость разработки одной скважины: 10 млрд рублей / 200 скважин = 50 млн рублей/скважина
* Доход с одного барреля (в тысячах): 450 тыс. рублей
* Безубыточный объем сырья (в тысячах баррелей): 50 млн рублей / 450 тыс. рублей/тыс. баррелей ≈ 111.11 тыс. баррелей

In [11]:
region_points = 500
num_wells = 200
budget = 10000000000
one_barrel = 450
income_per_barrel = 450000

cost_per_well = budget / num_wells
product = round(cost_per_well / income_per_barrel, 2)
print(product)

111.11


In [12]:
preds = [predictions_0.mean(), predictions_1.mean(), predictions_2.mean()]
for i in enumerate(preds):
    if i[1] > product:
        print(f'Region {i[0]} - Достаточно')
    else:
        print(f'Region {i[0]} - Недостаточно')

Region 0 - Недостаточно
Region 1 - Недостаточно
Region 2 - Недостаточно


Все ключевые значения для расчётов сохранили в отдельных переменных и рассчитали безубыточный объем сырья. Предсказанного среднего запаса сырья в трех регионах оказалось недостаточным для безубыточной разработки

## Расчёт прибыли и рисков 

**Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:**

 4.1. Выберите скважины с максимальными значениями предсказаний. 
 
 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
 
 4.3. Рассчитайте прибыль для полученного объёма сырья.
 
**Посчитайте риски и прибыль для каждого региона:**
 
 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
 
 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
 
 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [17]:
def calculate_profit(data, budget, cost_per_well, income_per_barrel, num_wells_to_develop):
    data_sorted = data.sort_values('preds', ascending=False)
    top_wells = data_sorted.head(num_wells_to_develop)
    total_volume = top_wells['product'].sum()
    total_cost = cost_per_well * num_wells_to_develop
    total_income = total_volume * income_per_barrel
    profit = total_income - total_cost
    return profit

profit = calculate_profit(results_2, budget, cost_per_well, income_per_barrel, num_wells)
print(f"Прибыль: {profit}")

Прибыль: 2598571759.374111


In [19]:
def calculate_risk(data, budget, cost_per_well, income_per_barrel, num_wells_to_develop, n_bootstrap_samples=1000):
    state = np.random.RandomState(12345)
    values = []
    for i in range(n_bootstrap_samples):
        bootstrap_sample = data.sample(n=500, replace=True, random_state=state)
        bootstrap_sample = bootstrap_sample.reset_index(drop=True) 

        profit = calculate_profit(bootstrap_sample, budget, cost_per_well, income_per_barrel, num_wells_to_develop)
        values.append(profit)

    values = pd.Series(values)
    mean_profit = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk_of_loss = (values < 0).mean()

    return {'mean_profit': mean_profit, 'confidence_interval': (lower, upper), 'risk_of_loss': risk_of_loss}

print('Geo_date0')
print(calculate_risk(
        results_0, budget, cost_per_well, income_per_barrel, num_wells))

print('Geo_date1')
print(calculate_risk(
        results_1, budget, cost_per_well, income_per_barrel, num_wells))


print('Geo_date2')
print(calculate_risk(
        results_2, budget, cost_per_well, income_per_barrel, num_wells))

Geo_date0
{'mean_profit': 406278783.42441905, 'confidence_interval': (-117742136.49486831, 911737050.7514055), 'risk_of_loss': 0.067}
Geo_date1
{'mean_profit': 432624131.8131374, 'confidence_interval': (16846174.932430126, 815972526.2857513), 'risk_of_loss': 0.019}
Geo_date2
{'mean_profit': 377362192.4229165, 'confidence_interval': (-170780417.7057271, 901772131.3864455), 'risk_of_loss': 0.074}


Geo_date1 выглядит наиболее привлекательным регионом для инвестиций. У него самая высокая прибыль среди остальных регионов и наименьший риск убытков (1.9%). Кроме того, нижняя граница его доверительного интервала (16.8 млн рублей) значительно выше нуля.

Geo_date0 имеет немного меньшую среднюю прибыль (406.3 млн рублей), чем Geo_date1, но существенно больший риск убытков (6.7%). Доверительный интервал для этого региона включает отрицательные значения.

Geo_date2 показывает наименьшую среднюю прибыль (377.4 млн рублей) и самый высокий риск убытков (7.4%).

**Вывод:**

1. Загрузили файлы в переменные и изучили данные о трех регионах.
2. Обучили модель Линейной Регрессии и посчитали метрику RMSE. Сохранили предсказанные значения, правильные ответы на валидационной выборке для каждого региона.
3. Посчитали минимальное среднее количество продукта в месторождениях региона, достаточное для разработки. Безубыточный объем сырья: 111.11 тыс. баррелей. Предсказанного среднего запаса сырья в трех регионах оказалось недостаточным для выгодной разработки
4. Провели процедуру Bootstrap, посчитали для каждого региона среднюю прибыль, 95%-й доверительный интервал и риск убытков. 
5. Получили, что Geo_date1 кажется наиболее прибыльным регионом для разработки. Он демонстрирует самую высокую среднюю прибыль (432.6 млн рублей) и наименьший риск убытков (1.9%) среди других регионов. 